<a href="https://colab.research.google.com/github/danski3456/coding_exercises/blob/main/exercises/Price%20Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nv https://raw.githubusercontent.com/danski3456/notebook_grading/main/utils.py -O utils.py
%run utils.py

!pip install eccd_datasets > /dev/nul
!pip install category_encoders > /dev/nul

In [ ]:
STUDENT_NAME = "Nombre Apellido"
COURSE_NAME = "eccd-feb22"
EXERCISE_NAME = "price-optimization"

# Objective

The objective of this notebook, is to learn how to perform price optimization.

The basic idea behind price optimization is to derive a model for demand forecasting that depends on the price, and then use the learned model to predict a demand curve.

For this we will use the M5 competition dataset, large and popular dataset.

In [ ]:
import pandas as pd
import numpy as np

from IPython.display import display

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
import statsmodels.tsa.arima_model as ARIMA

from category_encoders import TargetEncoder

import xgboost as xgb
from eccd_datasets import load_m5

In [ ]:
datasets = load_m5()
df_calendar = datasets["calendar"]
df_sales = datasets["sales"]
df_prices = datasets["sell_prices"]

In [ ]:
def get_data_from_id(id_: str, sales: pd.DataFrame, prices: pd.DataFrame, calendar: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts the dataframe associated with a single item id in long format.
    """
    
    df = sales[sales["id"] == id_].drop(columns=["id"])
    
    df = pd.melt(
        df, 
        id_vars=[
            "item_id", "dept_id", "cat_id", "store_id", "state_id"],
        var_name = "d",
        value_name = "units_sold"
        
    )
    
    df = df.merge(calendar, on="d", how="left")
    
    df = df.merge(prices, on=["item_id", "store_id", "wm_yr_wk"], how="left")
    
    df["date"] = pd.to_datetime(df["date"])
    
    return df
    

In [ ]:
ID = "HOBBIES_1_001_CA_1_validation"
df_id = get_data_from_id(ID, df_sales, df_prices, df_calendar)
df_id.head()

# Price variations

To be able to derive a demand curve, we need a model that depends on prices. 

This implies that we should have several price chances in a product if we want to have any chance of learning a good model.

In [ ]:
def get_id_with_most_price_changes(sales: pd.DataFrame, prices: pd.DataFrame) -> str:
    """
    This function returns the ids of all the rows that are tied with the highest
    number of price changes.
    """
    
    # Write your code here
    

In [ ]:
answer_number_ids = get_id_with_most_price_changes(df_sales, df_prices)
print(answer_number_ids)

In [ ]:
dfs = []
for id_ in answer_number_ids:
    df = get_data_from_id(id_, df_sales, df_prices, df_calendar)
    dfs.append(df)
    
df = pd.concat(dfs)

In [ ]:
df = df.drop(columns=[
    "item_id", "dept_id", "cat_id", "state_id", "d", "wm_yr_wk",
    "event_name_1", "event_type_1", "event_name_2", "event_type_2",
    "date"
])

In [ ]:
y = df.pop("units_sold")
X = df.copy()

In [ ]:
te = TargetEncoder(cols=["store_id", "weekday"])
X = te.fit_transform(X, y)

In [ ]:
model = xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=10000,
        random_state=4,
        max_depth=6,
        booster="gbtree",
        learning_rate=0.01,
        subsample=0.7,
        gamma=0,
        reg_alpha=0,
        reg_lambda=1,
        importance_type="gain",
        colsample_bytree=0.9,
        colsample_bynode=0.8,
        colsample_bylevel=0.8,
        verbosity=1,
        n_jobs=1,
)

model.fit(X, y)

In [ ]:
def generate_demand_curve(min_price: float, max_price: float, n_points: int, df: pd.DataFrame):
    """
    This function duplicates a row in the original dataset, but changes the price
    to generate new predictions
    """
    
    prices = np.linspace(min_price, max_price, n_points)
    
    sample_row = df.iloc[-1, :]
    
    data = []
    for p in prices:
        row = sample_row.copy()
        row["sell_price"] = p
        data.append(row)
        
    new_df = pd.concat(data, axis=1)
    return new_df.T, prices

In [ ]:
tmp_df, prices = generate_demand_curve(3, 5, 20, X)

In [ ]:
predicted_units = model.predict(tmp_df)

# Unintuitive demand curves

If we look at the plot, we can see that the demand increases with the price.

This goes against our intuitive knowledge of how markets and demand work.

We can fix this issue by adding additional biases to the model.

In [ ]:
plt.plot(prices, predicted_units)

In [ ]:
def build_monotonic_xgb(df: pd.DataFrame) -> xgb.XGBRegressor:
    """
    Implement a function that builds an XGBoost model with the property that
    for any two rows `r1` and `r2` if `r1["sel_price"] > r2["sell_price"]` then
    it holds that the predicted sold units for `r1` are smaller or equal than the
    predicted sold units for `r2`.
    """
    
    # Write your code here
    
    model = xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=10000,
        random_state=4,
        max_depth=6,
        booster="gbtree",
        learning_rate=0.01,
        subsample=0.7,
        gamma=0,
        reg_alpha=0,
        reg_lambda=1,
        importance_type="gain",
        colsample_bytree=0.9,
        colsample_bynode=0.8,
        colsample_bylevel=0.8,
        verbosity=1,
        n_jobs=1,
        monotone_constraints = tuple(monotone_constraints)
    )

    

    return model

In [ ]:
monotonic_model = build_monotonic_xgb(X)
monotonic_model.fit(X, y)
y_pred = monotonic_model.predict(tmp_df)

In [ ]:
y_pred

In [ ]:
answer_negative_diffs = np.diff(y_pred) <= 0
print(answer_negative_diffs)

We managed to get a monotinic model that is consistent with our expectations of reality. 

Still, there are problems with this basic model. For instance, it returns negative demand units, which is not possible.

In further iteration, we should constraint the model to only return non-negative results.

In [ ]:
fig, ax = plt.subplots()

ax.plot(prices, y_pred)
ax.axhline(y=0, c="r")

In [ ]:

proposed_solution = {
'attempt': {
    'course_name': COURSE_NAME,
    'exercise_name': EXERCISE_NAME,
    'username': STUDENT_NAME,
},
'task_attempts': [
	{
		"name": "strictly-negative",
		"answer": str(answer_negative_diffs),
	},
	{
		"name": "maximum-price-changes",
		"answer": str(answer_number_ids),
	},
]

}
check_solution(proposed_solution)
    